In [ ]:
import whisper
import moviepy.editor as mp
import json
import datetime
import pandas as pd
import os
from googletrans import Translator
#from stable_whisper import modify_model

In [ ]:
dir = 'failed_videos'

# make list of files
files = os.listdir(dir)
print(files)
model = whisper.load_model('medium')

In [ ]:
def format_data(english_result, original_result, video_id):
    '''
    THIS WILL CHANGE TO FORMAT INTO CSV FORMAT
    '''
    output = {}

    # build dictionary
    output['Content ID'] = video_id
    output['English Transcription'] = english_result
    output['Original Language Transcription'] = original_result
    return output

In [ ]:
csv_list = []
translator = Translator()

for file in files:
    file_path = f"{dir}/{file}"
    video_file = file_path
    audio_file = os.path.basename(file_path)
    audio_file = f"{os.path.splitext(audio_file)[0]}.wav"
    
    # load video
    clip = mp.VideoFileClip(video_file)
    if clip.audio is None:
        print(f'NO AUDIO DETECTED: {video_file}')
        continue

    # split audio from video
    clip.audio.write_audiofile(audio_file)

    # detect language
    audio = whisper.load_audio(audio_file)
    audio = whisper.pad_or_trim(audio)
    mel = whisper.log_mel_spectrogram(audio).to(model.device)
    _, probs = model.detect_language(mel)
    language = max(probs, key=probs.get)
    print(language)
    
    # if language confidence is low, skip transcription
    if probs.get(language) < 0.1:
        print("language confidence too low, quitting...")
        os.remove(audio_file)
        original_result = None
        english_result = None

    # if language not english, translate to english
    if language != 'en':
        print('language not english, transcribing original language...')
        original_result = model.transcribe(audio_file, verbose=True)
        print('translating to english...')
        english_result = ""
        for segment in original_result['segments']:
            english = translator.translate(segment['text'])
            english = english.text
            english_result += english

        original_result = original_result['text']
    else:
        print('language is english, transcribing...')
        #english_result = model.transcribe(audio_file, beam_size=5, best_of=5, verbose=True)
        english_result = model.transcribe(audio_file, verbose=True)
        english_result = english_result['text']
        original_result = None

    # format data for web
    output_data = format_data(english_result, original_result, os.path.splitext(file)[0])
    csv_list.append(output_data)
    
    # remove audio file
    os.remove(audio_file)
    print(file_path, ' completed')

In [ ]:
df = pd.DataFrame(csv_list)
df.to_csv('peru_transcriptions.csv')